<a href="https://colab.research.google.com/github/sruby8/trackmanreports/blob/main/trackman_postgame_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Reusable Pro Scouting Report Generator (Python) ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF
from PIL import Image

# ---------------------------
# SETTINGS
# ---------------------------

# Input CSV file (exported TrackMan summary or custom table)
input_csv = "your_trackman_data.csv"  # <-- Change this to your file
output_pdf = "scouting_report_output.pdf"

# Chart theme settings
plt.style.use('dark_background')
pitch_colors = {
    "Four-Seam": "#FF4136",
    "Slider": "#0074D9",
    "Sinker": "#FFDC00",
    "Splitter": "#2ECC40",
    "Changeup": "#B10DC9"
}

# ---------------------------
# LOAD DATA
# ---------------------------

# Minimal example dataset structure
df = pd.read_csv(input_csv)

# Expected columns: Pitch Type, Velo, Spin Rate, IVB, HB

# Simulate pitch-by-pitch sequence if needed (for trends)
pitch_types_sequence = list(df['Pitch Type'].values)
velo_sequence = list(df['Velo'].values)
spin_sequence = list(df['Spin Rate'].values)

# ---------------------------
# BUILD CHARTS
# ---------------------------

# Movement Chart
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_facecolor('black')
for pitch in df['Pitch Type'].unique():
    mask = df['Pitch Type'] == pitch
    ax.scatter(df.loc[mask, 'HB'], df.loc[mask, 'IVB'],
               s=120, color=pitch_colors.get(pitch, 'white'), edgecolors='white', label=pitch)
ax.axhline(0, color='gray', lw=1)
ax.axvline(0, color='gray', lw=1)
ax.set_xlabel('Horizontal Break (inches)', color='white')
ax.set_ylabel('Induced Vertical Break (inches)', color='white')
ax.legend(facecolor='black', edgecolor='white', fontsize=8)
plt.tight_layout()
movement_chart_path = "movement_chart.png"
plt.savefig(movement_chart_path, dpi=300, bbox_inches='tight')
plt.close()

# Heatmap (simulated since TrackMan location data may not be provided)
x = np.random.normal(0, 0.5, 100)
y = np.random.normal(2.5, 0.7, 100)
fig, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(x=x, y=y, cmap="rocket", fill=True, thresh=0.05, levels=100, alpha=0.8)
ax.axhline(0, color='gray')
ax.axvline(0, color='gray')
ax.set_xlim(-2, 2)
ax.set_ylim(0.5, 4.5)
ax.set_xlabel('Plate Width (feet)', color='white')
ax.set_ylabel('Height (feet)', color='white')
plt.tight_layout()
heatmap_path = "heatmap_chart.png"
plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
plt.close()

# Velocity Trend
fig, ax = plt.subplots(figsize=(8, 3))
ax.set_facecolor('black')
ax.plot(range(1, len(velo_sequence)+1), velo_sequence, color='white', linestyle='--')
ax.scatter(range(1, len(velo_sequence)+1), velo_sequence, c='red', edgecolors='white')
ax.set_xlabel('Pitch #', color='white')
ax.set_ylabel('Velocity (mph)', color='white')
plt.tight_layout()
velo_trend_path = "velo_trend.png"
plt.savefig(velo_trend_path, dpi=300, bbox_inches='tight')
plt.close()

# Spin Rate Trend
fig, ax = plt.subplots(figsize=(8, 3))
ax.set_facecolor('black')
ax.plot(range(1, len(spin_sequence)+1), spin_sequence, color='white', linestyle='--')
ax.scatter(range(1, len(spin_sequence)+1), spin_sequence, c='blue', edgecolors='white')
ax.set_xlabel('Pitch #', color='white')
ax.set_ylabel('Spin Rate (rpm)', color='white')
plt.tight_layout()
spin_trend_path = "spin_trend.png"
plt.savefig(spin_trend_path, dpi=300, bbox_inches='tight')
plt.close()

# ---------------------------
# BUILD SCOUTING REPORT PDF
# ---------------------------

class ScoutingPDF(FPDF):
    def header(self):
        self.set_fill_color(0, 0, 0)
        self.rect(0, 0, 210, 297, 'F')
        self.set_font('Arial', 'B', 20)
        self.set_text_color(255, 255, 255)
        self.cell(0, 10, 'Pro Scouting Report', ln=True, align='C')
        self.ln(5)

    def section_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.set_text_color(0, 255, 255)
        self.cell(0, 8, title, ln=True, align='C')
        self.ln(2)

    def add_centered_image(self, path, width):
        x = (210 - width) / 2
        self.image(path, x=x, y=self.get_y(), w=width)
        self.ln(width * 0.45)

# Create PDF
pdf = ScoutingPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=False)

# Movement + Heatmap
pdf.section_title('Pitch Movement Chart (Left) + Pitch Location Heatmap (Right)')
pdf.image(movement_chart_path, x=10, y=pdf.get_y(), w=85)
pdf.image(heatmap_path, x=110, y=pdf.get_y(), w=85)
pdf.ln(85)

# Arsenal Summary (simple table)
pdf.ln(20)
pdf.section_title('Pitch Arsenal Summary')
pdf.set_font('Arial', '', 8)
for idx, row in df.iterrows():
    line = f"{row['Pitch Type']}: Velo={row['Velo']} mph | Spin={row['Spin Rate']} rpm | IVB={row['IVB']} | HB={row['HB']}"
    pdf.cell(0, 6, line, ln=True, align='C')

# Page 2 - Trends
pdf.add_page()
pdf.set_y(20)
pdf.section_title('Velocity Trend')
pdf.add_centered_image(velo_trend_path, width=160)

pdf.section_title('Spin Rate Trend')
pdf.add_centered_image(spin_trend_path, width=160)

# Save Final PDF
pdf.output(output_pdf)

print(f"✅ Scouting report generated: {output_pdf}")


ModuleNotFoundError: No module named 'fpdf'